# 521153S Deep Learning Final Project

In [28]:
# import necessary packages
import os
import requests
import zipfile
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.io import read_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights, vgg11, vgg13, vgg16, vgg19, VGG11_Weights, VGG13_Weights, VGG16_Weights, VGG19_Weights
import gdown
import urllib
import os
import random
import tarfile
import time
import shutil
import logging as lg
from tqdm.notebook import tqdm

In [2]:
# Set the logging level
# Possible Levels: DEBUG, INFO,
lg.basicConfig(level=lg.DEBUG)

In [3]:
# download the dataset
val_url = 'https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl'
train_url = 'https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs'
test_url = 'https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v'
eurosat_url = 'https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1'

val_file = './data/val.tar'
train_file = './data/train.tar'
test_file = './data/test.tar'
eurosat_file = './data/eurosatrgb.zip'


if not os.path.exists('./data'):
    print('Creating data directory')
    os.mkdir('./data')

if not os.path.exists('./data/val.tar'):
    print('Downloading val.tar')
    gdown.download(val_url, val_file)
    val_tar = tarfile.open(val_file)
    val_tar.extractall('./data/')
    val_tar.close()

if not os.path.exists(train_file):
    print('Downloading train.tar')
    gdown.download(train_url, train_file)
    train_tar = tarfile.open(train_file)
    train_tar.extractall('./data/')
    train_tar.close()

if not os.path.exists(test_file):
    print('Downloading test.tar')
    gdown.download(test_url, test_file)
    test_tar = tarfile.open(test_file)
    test_tar.extractall('./data/')
    test_tar.close()

if not os.path.exists(eurosat_file):
    print('Downloading EuroSAT_RGB.zip')
    response = urllib.request.urlretrieve(eurosat_url, eurosat_file)
    eurosat_zip = zipfile.ZipFile(eurosat_file)
    eurosat_zip.extractall('./data/eurosat')
    eurosat_zip.close()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): drive.google.com:443


Creating data directory


DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl&confirm=t&uuid=c0b0f627-c14b-4488-9be4-5aecaadb2dcf HTTP/1.1" 303 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-00-3g-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-00-3g-docs.googleusercontent.com:443 "GET /docs/securesc/0kmt8bdmqm98llfgnreft9bpc75s23vp/3g2uarrf6gmeaaa9qocipvmivu09rtg2/1702652175000/12865399289486813135/14485584017585893037Z/1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl?uuid=c0b0f627-c14b-4488-9be4-5aecaadb2dcf HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /nonceSigner?nonce=8u91kof0vjoqg&continue=https://doc-00-3g-docs.googleusercontent.com/docs/securesc

DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs&confirm=t&uuid=9d4124c8-dd39-4886-9911-8f23bed5eff2 HTTP/1.1" 303 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-0g-3g-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-0g-3g-docs.googleusercontent.com:443 "GET /docs/securesc/0kmt8bdmqm98llfgnreft9bpc75s23vp/5oht94nk6avr0h99bost51bvncbsb1t4/1702652175000/12865399289486813135/14485584017585893037Z/107FTosYIeBn5QbynR46YG91nHcJ70whs?uuid=9d4124c8-dd39-4886-9911-8f23bed5eff2 HTTP/1.1" 200 125900800
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs
From (redirected): https://drive.google.com/uc?id=107F

DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v HTTP/1.1" 303 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-10-3g-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-10-3g-docs.googleusercontent.com:443 "GET /docs/securesc/0kmt8bdmqm98llfgnreft9bpc75s23vp/m2o9lra63ak58osgc0j9d35flgpffvf9/1702652175000/12865399289486813135/14485584017585893037Z/1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v?uuid=45ea8466-3916-4123-8970-a83e92985639 HTTP/1.1" 200 39178240
Downloading...
From: https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v
To: d:\Koodi\Deep-Learning-Project\data\test.tar
100%|██████████| 39.2M/39.2M [00:03<00:00, 11.5MB/s]


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')
print('Device:', device)

Device: cuda:0


In [19]:
# Image Size
image_size = 32

# Batch size during training
batch_size = 100
num_workers = 1

# Learning rate for optimizers
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 30

# Weight decay
weight_decay=0.0001

In [7]:
model_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT)
model_resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
model_resnet50 = resnet50(weights=ResNet50_Weights.DEFAULT)
model_resnet101 = resnet101(weights=ResNet101_Weights.DEFAULT)
model_resnet152 = resnet152(weights=ResNet152_Weights.DEFAULT)

model_vgg11 = vgg11(weights=VGG11_Weights.DEFAULT)
model_vgg13 = vgg13(weights=VGG13_Weights.DEFAULT)
model_vgg16 = vgg16(weights=VGG16_Weights.DEFAULT)
model_vgg19 = vgg19(weights=VGG19_Weights.DEFAULT)

train_dataset = torchvision.datasets.ImageFolder(
    root='./data/train',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

test_dataset = torchvision.datasets.ImageFolder(
    root='./data/test',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

validation_dataset = torchvision.datasets.ImageFolder(
    root='./data/val',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)


dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader_validation = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [26]:
def save_model(trained_model):
    # Create Models Folder
    if not os.path.exists('./models'):
        print('Creating models directory')
        os.mkdir('./models')

    # Save the model
    if not os.path.exists(f'./models/{trained_model.name}.pth'):
        print(f'Saving Model {trained_model.name}')
        torch.save(trained_model, f'./models/{trained_model.name}.pth')

def load_model(model_name):
    if not os.path.exists(f'./models/{model_name}.pth'):
        print(f'Cannot Load {model_name}')
        return None
    print(f'Loading Model {model_name}')
    model = torch.load(f'.models/{model_name}.pth')
    model.eval()
    return model

In [9]:
# Define the used model
model = model_resnet18
model.name = "resnet18"     # Set name for saving

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr, momentum)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [14]:
def evaluate(model, data_loader, criterion, device):

    model.eval() 
    # valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    num_images = 0.0 # used to accumulate number of images
    test_acc = 0
    
    #with torch.no_grad():
    for i, data in enumerate(data_loader, 0):
        counter += 1
        
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        # loss = criterion(outputs, labels)
        # valid_running_loss += loss.item()
        # Calculate the accuracy.
        #preds = outputs.argmax(dim=1)
        preds = torch.max(outputs, 1).indices
        test_acc += int(torch.sum(preds == labels))
        
        #valid_running_correct += preds.eq(labels).sum()
        valid_running_correct += (preds == labels).sum().item()
        num_images += len(labels)
        
    # Loss and accuracy for the complete epoch.
    # epoch_loss = valid_running_loss / counter
    print("Number of correct predictions: ", valid_running_correct)

    epoch_acc = test_acc / num_images
    return epoch_acc

In [15]:
def train(model, dataloader_train, criterion, optimizer, num_epochs, device):
    model.to(device)
    train_loss = []
    train_acc = []

    # Start the training.
    if torch.cuda.is_available():
        print("Using CUDA")

    print('Training')
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        train_running_correct = 0
        counter = 0
        num_images = 0.0 # used to accumulate number of images
        for i, data in enumerate(dataloader_train,0):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += (preds == labels).sum().item()
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()    # Output training 
            num_images += len(image)
            if counter % 100 == 0:
                print(f'Epoch: {epoch+1}/{num_epochs}, Step: {counter}/{len(dataloader_train)}, Loss: {loss.item():.4f}')


        # Validation
        valid_epoch_acc = evaluate(model, dataloader_validation, criterion, device)
        

        lg.debug(f"counter: {counter}, len(train_dataset): {len(train_dataset)}")
        lg.debug(f"train_running_loss: {train_running_loss}, train_running_correct: {train_running_correct}")
        lg.debug(f"Loss.item: {loss.item()}")
        lg.debug(f"Loss: {loss}")
        
        train_epoch_loss = train_running_loss / counter
        train_loss.append(train_epoch_loss)
        # valid_loss.append(valid_epoch_loss)
        train_epoch_acc = train_running_correct/len(train_dataset)
        train_acc.append(train_epoch_acc )

        print('Epoch: %d/%d, Training accuracy: %f, loss: %f, validation accuracy: %f' % (epoch+1, num_epochs, train_epoch_acc, loss, valid_epoch_acc))

        # valid_acc.append(valid_epoch_acc)
        # print(f"Training loss mean: {train_epoch_loss :.3f}, training acc: {train_epoch_acc:.3f}")
        # print(f"Validation acc: {valid_epoch_acc:.3f}")
        print('-'*50)


    return model

In [31]:
trained_model = train(model, dataloader_train, criterion, optimizer, num_epochs, device)

Using CUDA
Training
Epoch: 1/1, Step: 100/384, Loss: 0.3665
Epoch: 1/1, Step: 200/384, Loss: 0.3072
Epoch: 1/1, Step: 300/384, Loss: 0.2282


DEBUG:root:counter: 384, len(train_dataset): 38400
DEBUG:root:train_running_loss: 137.0183524787426, train_running_correct: 34245
DEBUG:root:Loss.item: 0.346039742231369
DEBUG:root:Loss: 0.346039742231369


Number of correct predictions:  117
Epoch: 1/1, Training accuracy: 0.891797, loss: 0.346040, validation accuracy: 0.012188
--------------------------------------------------


In [32]:
save_model(trained_model)

In [ ]:
# validate_loss, validate_acc = evaluate(trained_model, dataloader_validation, criterion, device)
# print(f"Validation loss: {validate_loss:.3f}, validation acc: {validate_acc:.3f}")
test_acc = evaluate(trained_model, dataloader_test, criterion, device)
print(f"test acc: {test_acc:.3f}")


In [ ]:
#NEWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
def evaluate(model, data_loader, device):

    model.eval() 
    print('Evaluate')
    valid_running_correct = 0.0
    num_images = 0.0 # used to accumulate number of images
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(data_loader, 0):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # Calculate the accuracy.
            #preds = outputs.argmax(dim=1)
            preds = torch.max(outputs,1).indices
            #valid_running_correct += preds.eq(labels).sum()
            valid_running_correct += int(torch.sum(preds == labels))
            num_images += len(labels)
        
        # accuracy for the complete epoch.
        acc = valid_running_correct / num_images
    return acc

In [ ]:
#NEWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
def train(model, dataloader_train, dataloader_validation, criterion, optimizer, num_epochs, device):
    model.to(device)
    train_loss = []
    train_acc = []

    # Start the training.
    if torch.cuda.is_available():
        print("Using CUDA")

    print('Training')
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0 # loss
        train_running_correct = 0.0 # accuracy
        num_images = 0.0 # used to accumulate number of images

        for i, data  in enumerate(dataloader_train,0):
            optimizer.zero_grad()
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass.
            outputs = model(images)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step() # Output training 

            # Calculate the accuracy.
            num_images += images.size(0)
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += (preds == labels).sum().item()

        acc = train_running_correct / num_images      
        acc_eval = evaluate(model, dataloader_validation, device)  
        print('epoch: %d, accuracy: %f, loss: %f, validation accuracy: %f' % (epoch, acc, loss, acc_eval))

    return model

In [ ]:
#NEWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
trained_model = train(model, dataloader_train, dataloader_validation, criterion, optimizer, num_epochs, device)

In [ ]:
# Choose 100 images from EuroSAT dataset

if not os.path.exists('./data/eurosat_selected'):
    print('Creating data directory')
    os.mkdir('./data/eurosat_selected')

if not os.path.exists('./data/eurosat_training'):
    print('Creating training directory')
    os.mkdir('./data/eurosat_training')

# Load the EuroSAT Categories
eurosat_categories = [name for name in os.listdir('./data/eurosat/EuroSAT_RGB/') if os.path.isdir(os.path.join('./data/eurosat/EuroSAT_RGB/', name))]
# Randomly select 5 categories
print(f"Selecting 5 categories from {eurosat_categories} categories")
selected_categories = np.random.choice(eurosat_categories, 5, replace=False)
print(f"Selected categories: {selected_categories}")

selected_images = []
training_images = []

# From each directory, randomly select 20 images
for category in selected_categories:
    images = os.listdir(os.path.join('./data/eurosat/EuroSAT_RGB/', category))
    selected = random.sample(images, 20)
    selected_images.extend([(category, image) for image in selected])
    print(f"Selected {selected} from {category}")
    # Copy selected images to the selected directory
    for image in selected:
        shutil.copyfile(os.path.join('./data/eurosat/EuroSAT_RGB', category, image), os.path.join('./data/eurosat_selected', image))

# From these 100 images, randomly select 5 images from each category for the training set
for category in selected_categories:
    category_images = [image for (cat, image) in selected_images if cat == category]
    training = random.sample(category_images, 5)
    training_images.extend(training)
    print(f"Selected {training} for training")

    # Copy training images to the training directory
    for image in training:
        shutil.copyfile(os.path.join('./data/eurosat_selected', image), os.path.join('./data/eurosat_training', image))